In [1]:
import glob
import SimpleITK as sitk
import numpy as np
import collections
from PIL import Image, ImageDraw
#from segmentation import segment
import os
import csv
import ast
import collections
from collections import namedtuple
import random
from pylidc_func import create_ann_malcsv, create_ann_maldf, masks_build
#from cmask_dict import cmask_dict
import torch

In [2]:
isLungSegmentProcess = False

In [3]:
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])

def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))


In [4]:
CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, series_uid, center_xyz, cbbox, mask_box')
def getCandidateInfoList(requireOnDisk_bool=True):
    # We construct a set with all series_uids that are present on disk.
    # This will let us use the data, even if we haven't downloaded all of
    # the subsets yet.
    mhd_list = glob.glob('D:/IMProject/LunaData/subset*/*.mhd')
    # mhd_list = glob.glob('D:/LUNA/Luna16_AugSegData/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    candidateInfo_list = []
    with open('./annotations_object_detection_1127.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            # annotationDiameter_mm = float(row[4])
            isMal_bool = {'0.0': False, '1.0': True}[row[5]] #it record the malignancy or not
            cbbox = ast.literal_eval(row[7])
            mask_box = row[8]
            mask_list = ast.literal_eval(mask_box)
            
            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            candidateInfo_list.append(
                CandidateInfoTuple(
                    True,
                    True,
                    isMal_bool,
                    series_uid,
                    annotationCenter_xyz,
                    cbbox,
                    mask_list
                )
            )
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list


In [5]:
candidateInfo_list = getCandidateInfoList()
# len(candidateInfo_list)

In [6]:
# candidateInfo_list[0]

In [7]:
mal_count_list = {'Benign': 0, 'Malignancy': 0, 'nan': 0}
for candidate in candidateInfo_list:
    if candidate[2]:
        for item in candidate[6]:
            if item == 'nan':
                mal_count_list['nan'] += 1
            else:
                mal_count_list['Malignancy'] += 1
    else:
         for item in candidate[6]:
            if item == 'nan':
                mal_count_list['nan'] += 1
            else:
                mal_count_list['Benign'] += 1

print(mal_count_list)

generate = {'Train': {'Benign': mal_count_list['Benign'] - 2 * int(mal_count_list['Benign'] * 0.1), 'Malignancy': mal_count_list['Malignancy'] - 2 * int(mal_count_list['Malignancy'] * 0.1)}, 'Valid': {'Benign':  int(mal_count_list['Benign'] * 0.1), 'Malignancy': int(mal_count_list['Malignancy'] * 0.1)}, 'Test': {'Benign':  int(mal_count_list['Benign'] * 0.1), 'Malignancy':  int(mal_count_list['Malignancy'] * 0.1)}}
print(generate)

{'Benign': 4117, 'Malignancy': 3788, 'nan': 966}
{'Train': {'Benign': 3295, 'Malignancy': 3032}, 'Valid': {'Benign': 411, 'Malignancy': 378}, 'Test': {'Benign': 411, 'Malignancy': 378}}


In [8]:
def buildAnnotationMask(noduleInfo_list, hu_a, origin_xyz, vxSize_xyz, direction_a, threshold_hu = -700):
        boundingBox_a = np.zeros_like(hu_a, dtype=bool) # all False tensor
        bbox_result = []
        for candidateInfo_tup in noduleInfo_list: #traverse所有的nodules
            center_irc = xyz2irc(
                candidateInfo_tup.center_xyz,
                origin_xyz,
                vxSize_xyz,
                direction_a,
            )
            ci = int(center_irc.index)  # the center of voxel
            cr = int(center_irc.row)
            cc = int(center_irc.col)

            index_radius = 2
            try:
                # 從index找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci + index_radius, cr, cc] > threshold_hu and \
                        hu_a[ci - index_radius, cr, cc] > threshold_hu:
                    index_radius += 1
            except IndexError:
                index_radius -= 1

            row_radius = 2
            try:
                # 從row找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci, cr + row_radius, cc] > threshold_hu and \
                        hu_a[ci, cr - row_radius, cc] > threshold_hu:
                    row_radius += 1
            except IndexError:
                row_radius -= 1

            col_radius = 2
            try:
                # 從column找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci, cr, cc + col_radius] > threshold_hu and \
                        hu_a[ci, cr, cc - col_radius] > threshold_hu:
                    col_radius += 1
            except IndexError:
                col_radius -= 1

            # assert index_radius > 0, repr([candidateInfo_tup.center_xyz, center_irc, self.hu_a[ci, cr, cc]])
            # assert row_radius > 0
            # assert col_radius > 0

            boundingBox_a[
                 ci - index_radius: ci + index_radius + 1,
                 cr - row_radius: cr + row_radius + 1,
                 cc - col_radius: cc + col_radius + 1] = True #將box裡的所有格子設成TRUE
            
            bbox_result.append([round((abs(cr) + 1) / 512, 10), round((abs(cc) + 1) / 512, 10), round(2 * row_radius / 512, 10), round(2 * col_radius / 512, 10)])

        #mask_a = boundingBox_a & (hu_a > threshold_hu)  #最後會對box和threshold低於-700的值做and
        mask_a = boundingBox_a
        return mask_a, bbox_result

In [9]:
def getCandidateInfoDict(requireOnDisk_bool=True):  #把candidateInfoList包成Dict
    candidateInfo_list = getCandidateInfoList(requireOnDisk_bool)
    candidateInfo_dict = {}

    for candidateInfo_tup in candidateInfo_list:
        candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                      []).append(candidateInfo_tup)

    return candidateInfo_dict

In [10]:
candidate_dict =  getCandidateInfoDict()
len(candidate_dict) # 600

600

In [11]:
# print(candidate_dict)

In [12]:
# def np2Png(np_arr, target_path):
#     min_value = np.min(np_arr)
#     max_value = np.max(np_arr) if np.max(np_arr) > 0 else 1
#     scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
#     scaled_np_arr = scaled_np_arr.astype(np.uint8)
#     slices = Image.fromarray(scaled_np_arr, mode='L')
#     slices = slices.resize((640, 640))
#     slices.save(target_path, cmap='gray')

In [13]:
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from PIL import Image 
# from matplotlib.patches import Rectangle

# def np2MarkPng(np_arr, target_path, row, col):
#     min_value = np.min(np_arr)
#     max_value = np.max(np_arr)
#     scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
#     scaled_np_arr = scaled_np_arr.astype(np.uint8)
#     slices = Image.fromarray(scaled_np_arr, mode='L')
#     draw = ImageDraw.Draw(slices)
#     x1, y1, x2, y2 = col-10, row-10, col+10, row+10
#     draw.rectangle([x1, y1, x2, y2], outline="white", width=1)
#     slices.save(target_path)
#     im = Image.open(target_path)

#     # Create figure and axes
#     fig, ax = plt.subplots()

#     # Display the image
#     ax.imshow(im)

# def bboxOnImg(img_path, bbox):
#     [x, y, height, width] = bbox
    
#     x *= 640
#     y *= 640
#     height *= 640
#     width *= 640
    
#     im2 = Image.open(img_path)

#     # Create figure and axes
#     fig2, ax2 = plt.subplots()

#     # Display the image
#     ax2.imshow(im2)
#     size = im2.size
#     print(size)

#     # Create a Rectangle patch
#     rect = Rectangle((x-height/2, y - width/2), height, width, linewidth=1, edgecolor='r', facecolor='none')

#     # Add the patch to the Axes
#     ax2.add_patch(rect)

#     plt.show()

In [14]:
from PIL import Image, ImageDraw
def np2Png(np_arr, target_name):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    scaled_np_arr = np_arr * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slice_ori = Image.fromarray(scaled_np_arr, mode='L')
    slice_ori.save(target_name)

In [15]:
idx = 0
uid_ndcnt = 0

record = {'Train': {'Benign': 0, 'Malignancy': 0}, 'Valid': {'Benign':  0, 'Malignancy': 0}, 'Test': {'Benign':  0, 'Malignancy':  0}, 'Total': 0}

malig_rotation, benign_rotation = 0, 0

for uid in candidate_dict:
    mhd_path = glob.glob('D:/IMProject/LunaData/subset*/{}.mhd'.format(uid))

    ct_mhd = sitk.ReadImage(mhd_path)
    if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
        ct_mhd = ct_mhd[...,0]

    hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    mask = masks_build(uid, hu_a)
    # aug_result = hu_a
    # if isLungSegmentProcess:
    #     aug_result = segment(hu_a, model, device)

    origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
    vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
    direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)
    # mask_a, bbox_result = buildAnnotationMask(candidate_dict[uid], hu_a, origin_xyz, vxSize_xyz, direction_a)
    
    # print("bbox_result:", bbox_result)

    lower_bound = 0
    upper_bound = len(hu_a)
    current_slice = 0
    uid_ndcnt = 0

    for candidateInfo_tup in candidate_dict[uid]:
        center_irc = xyz2irc(candidateInfo_tup.center_xyz, origin_xyz, vxSize_xyz, direction_a)
        index, row, column = center_irc
        
        hu_list = []
        mask_list = []
        
        for i in range(candidateInfo_tup.cbbox[0], candidateInfo_tup.cbbox[1]):
            con_mask = candidateInfo_tup[6][i - candidateInfo_tup.cbbox[0]]
            #print("row: ", row, " column: ", column, " bbox: ", bbox_result[uid_ndcnt])
            # print(con_mask)
            if con_mask == 'nan':
                #print('found nan')
                continue
            #new_con_mask = [bbox_result[uid_ndcnt][0], bbox_result[uid_ndcnt][1], con_mask[2], con_mask[3]]
            new_con_mask = [con_mask[0], con_mask[1], con_mask[2], con_mask[3]]
            current_slice = i
            # current_slice = index + i
            # if index + i < lower_bound:
            #     current_slice = lower_bound
            # if index + i > upper_bound:
            #     current_slice = upper_bound
            
            new_center_irc = (current_slice, round(con_mask[1] * 512), round(con_mask[0] * 512))
            width_irc = (1, 64, 64)
            slice_list = []
            for axis, center_val in enumerate(new_center_irc):
                start_ndx = int(round(center_val - width_irc[axis]/2))
                end_ndx = int(start_ndx + width_irc[axis])

                # assert center_val >= 0 and center_val < self.hu_a.shape[axis], repr([self.series_uid, center_xyz, self.origin_xyz, self.vxSize_xyz, new_center_irc, axis])

                if start_ndx < 0:
                    # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                    #     self.series_uid, center_xyz, new_center_irc, self.hu_a.shape, width_irc))
                    start_ndx = 0
                    end_ndx = int(width_irc[axis])

                if end_ndx > hu_a.shape[axis]:
                    # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                    #     self.series_uid, center_xyz, new_center_irc, self.hu_a.shape, width_irc))
                    end_ndx = hu_a.shape[axis]
                    start_ndx = int(hu_a.shape[axis] - width_irc[axis])

                slice_list.append(slice(start_ndx, end_ndx))

            ct_chunk = hu_a[tuple(slice_list)]
            mask_chunk = mask[tuple(slice_list)]
            
            # np2Png(mask_chunk[0], "test.png")
            # a = input()
            
            label_t = torch.tensor([False, False], dtype=torch.float32)
            if not candidateInfo_tup.isMal_bool:
                label_t[0] = True
                # index_t = 0
            else:
                label_t[1] = True
                # index_t = 1
            
            gen_root_path = "D:/IMProject/DetectionData/"
            # gen_root_path = "C:/LUNA/object_detection/aug_lung_aug_new/"
            is_training = False
            doTest = True
            if candidateInfo_tup.isMal_bool:
                if generate['Train']['Malignancy'] > 0 and malig_rotation % 10 < 8:
                    gen_path = gen_root_path + 'Train'
                    generate['Train']['Malignancy'] -= 1
                    is_training = True
                    record['Train']['Malignancy'] += 6
                    record['Total'] += 5
                else:
                    if generate['Valid']['Malignancy'] > 0 and malig_rotation % 10 == 8:
                        gen_path = gen_root_path + 'Valid'
                        generate['Valid']['Malignancy'] -= 1
                        record['Valid']['Malignancy'] += 1
                        
                    else:
                        gen_path = gen_root_path + 'Test'
                        generate['Test']['Malignancy'] -= 1
                        record['Test']['Malignancy'] += 1
                        doTest = True
                malig_rotation += 1                
            else:
                if generate['Train']['Benign'] > 0 and benign_rotation % 10 < 8:
                    gen_path = gen_root_path + 'Train'
                    generate['Train']['Benign'] -= 1
                    is_training = True
                    record['Train']['Benign'] += 6
                    record['Total'] += 5
                else:
                    if generate['Valid']['Benign'] > 0 and benign_rotation % 10 == 8:
                        gen_path = gen_root_path + 'Valid'
                        generate['Valid']['Benign'] -= 1
                        record['Valid']['Benign'] += 1
                    else:
                        gen_path = gen_root_path + 'Test'
                        generate['Test']['Benign'] -= 1
                        record['Test']['Benign'] += 1
                        doTest = True
                benign_rotation += 1
            record['Total'] += 1
            
            if doTest:
                candidate_t = torch.from_numpy(ct_chunk).to(torch.float32)
                # print("org =", candidate_t.shape)
                torch.save(candidate_t, (gen_path + "/{}-({})-[{}]_{}_{}_ct.pt").format(idx, i, 0, uid_ndcnt, uid))
                mask_t = torch.from_numpy(mask_chunk).to(torch.float32)
                torch.save(mask_t, (gen_path + "/{}-({})-[{}]_{}_{}_mask.pt").format(idx, i, 0, uid_ndcnt, uid))
                torch.save(label_t, (gen_path + "/{}-({})-[{}]_{}_{}_label.pt").format(idx, i, 0, uid_ndcnt, uid))
                
                # np2Png(aug_result[current_slice], (gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 0, uid_ndcnt, uid))
                #np2Png(mask_a[current_slice].astype(float), "C:/LUNA/U-Det/Data/masks-photo/{}-({})_{}_{}.png".format(idx, i, uid_ndcnt, uid))
                #print("Ans:")
                #np2MarkPng(hu_a[current_slice], "E:/LUNA/Autolabel/comparison/{}-({})_{}_{}_true.png".format(idx, i, uid_ndcnt, uid), row, column)
                #print("result:")
                #bboxOnImg((gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 0, uid_ndcnt, uid), new_con_mask)            
                # with open((gen_path + "/labels/{}-({})-[{}]_{}_{}.txt").format(idx, i, 0, uid_ndcnt, uid), 'w') as txt:
                #     txt.write(str(int(candidateInfo_tup[2])) + ' ')
                #     for bbox_value in new_con_mask:
                #         txt.write(str(bbox_value) + ' ')
                
                if is_training:
                    for j in range(1, 4):
                        # print("j =", j)
                        # print(ct_chunk)
                        # print(np.rot90(ct_chunk, k = j))
                        candidate_t = torch.from_numpy(np.rot90(ct_chunk, k = j).copy()).to(torch.float32).squeeze().unsqueeze(0)
                        # print(f"rot {j} =", candidate_t.shape)
                        torch.save(candidate_t, (gen_path + "/{}-({})-[{}]_{}_{}_ct.pt").format(idx, i, j, uid_ndcnt, uid))
                        mask_t = torch.from_numpy(np.rot90(mask_chunk, k = j).copy()).to(torch.float32).squeeze().unsqueeze(0)
                        torch.save(mask_t, (gen_path + "/{}-({})-[{}]_{}_{}_mask.pt").format(idx, i, j, uid_ndcnt, uid))
                        torch.save(label_t, (gen_path + "/{}-({})-[{}]_{}_{}_label.pt").format(idx, i, j, uid_ndcnt, uid))
                        
                        # np2Png(np.rot90(aug_result[current_slice], k = j), (gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, j, uid_ndcnt, uid))
                        
#                         degrees = 90 * j
#                         [x_rotate, y_rotate, width_rotate, height_rotate] = new_con_mask
                        
#                         if degrees == 90:
#                             rotation_matrix = np.array([[0, 1], [-1, 0]])
#                         elif degrees == 180:
#                             rotation_matrix = np.array([[-1, 0], [0, -1]])
#                         elif degrees == 270:
#                             rotation_matrix = np.array([[0, -1], [1, 0]])
#                         else:
#                             raise ValueError("Invalid rotation degrees. Supported values are 90, 180, and 270.")

#                         # Rotate the rectangle's center (x, y)
#                         rotated_center = np.dot(rotation_matrix, np.array([x_rotate, y_rotate]))
#                         #print("rotate center: ", rotated_center)

#                         # Update the rectangle's position
#                         x_rotated, y_rotated = rotated_center
#                         if j == 1:
#                             rotate_new_con_mask = [x_rotated, 1 + y_rotated, height_rotate, width_rotate]
#                         elif j == 2:
#                             rotate_new_con_mask = [1 + x_rotated, 1 + y_rotated, width_rotate, height_rotate]
#                         elif j == 3:
#                             rotate_new_con_mask = [1 + x_rotated, y_rotated, height_rotate, width_rotate]
                            
                        # with open((gen_path + "/labels/{}-({})-[{}]_{}_{}.txt").format(idx, i, j, uid_ndcnt, uid), 'w') as txt:
                        #     txt.write(str(int(candidateInfo_tup[2])) + ' ')
                        #     for bbox_value in rotate_new_con_mask:
                        #         txt.write(str(bbox_value) + ' ')
                        
                        #bboxOnImg((gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, j, uid_ndcnt, uid), rotate_new_con_mask)            

                    
                    candidate_t = torch.from_numpy(np.flip(ct_chunk, axis=1).copy()).to(torch.float32)
                    # print("flip 1 =", candidate_t.shape)
                    torch.save(candidate_t, (gen_path + "/{}-({})-[{}]_{}_{}_ct.pt").format(idx, i, 4, uid_ndcnt, uid))
                    mask_t = torch.from_numpy(np.flip(ct_chunk, axis=1).copy()).to(torch.float32)
                    torch.save(mask_t, (gen_path + "/{}-({})-[{}]_{}_{}_mask.pt").format(idx, i, 4, uid_ndcnt, uid))
                    torch.save(label_t, (gen_path + "/{}-({})-[{}]_{}_{}_label.pt").format(idx, i, 4, uid_ndcnt, uid))
                    
                    # np2Png(np.flip(aug_result[current_slice], axis=1), (gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 4, uid_ndcnt, uid)) # horizontal
                    # rotate_new_con_mask = [1 - new_con_mask[0], new_con_mask[1], new_con_mask[2], new_con_mask[3]]
                    # with open((gen_path + "/labels/{}-({})-[{}]_{}_{}.txt").format(idx, i, 4, uid_ndcnt, uid), 'w') as txt:
                    #     txt.write(str(int(candidateInfo_tup[2])) + ' ')
                    #     for bbox_value in rotate_new_con_mask:
                    #         txt.write(str(bbox_value) + ' ')
                                
                    #bboxOnImg((gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 4, uid_ndcnt, uid), rotate_new_con_mask)
                    
                    
                    candidate_t = torch.from_numpy(np.flip(ct_chunk, axis=0).copy()).to(torch.float32)
                    # print("flip 0 =",candidate_t.shape)
                    torch.save(candidate_t, (gen_path + "/{}-({})-[{}]_{}_{}_ct.pt").format(idx, i, 5, uid_ndcnt, uid))
                    mask_t = torch.from_numpy(np.flip(mask_chunk, axis=0).copy()).to(torch.float32)
                    torch.save(mask_t, (gen_path + "/{}-({})-[{}]_{}_{}_mask.pt").format(idx, i, 5, uid_ndcnt, uid))
                    torch.save(label_t, (gen_path + "/{}-({})-[{}]_{}_{}_label.pt").format(idx, i, 5, uid_ndcnt, uid))
                    
                    # rotate_new_con_mask = [new_con_mask[0], 1 - new_con_mask[1], new_con_mask[2], new_con_mask[3]]
                    # np2Png(np.flip(aug_result[current_slice], axis=0), (gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 5, uid_ndcnt, uid)) # vertical
                    # with open((gen_path + "/labels/{}-({})-[{}]_{}_{}.txt").format(idx, i, 5, uid_ndcnt, uid), 'w') as txt:
                    #     txt.write(str(int(candidateInfo_tup[2])) + ' ')
                    #     for bbox_value in rotate_new_con_mask:
                    #         txt.write(str(bbox_value) + ' ')
                    
                    #bboxOnImg((gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 5, uid_ndcnt, uid), rotate_new_con_mask)

            # print(record)
            #hu_list.append(hu_a[current_slice])
            #mask_list.append(mask_a[current_slice])
        #tr_data = np.stack(hu_list)
        #gt_data = np.stack(mask_list)
        #np.save('C:/LUNA/U-Det/Data/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), tr_data)
        #np.save('C:/LUNA/U-Det/Data/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), gt_data)
        idx += 1
        uid_ndcnt += 1
print(record)

Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Failed to reduce all groups to <= 4 Annotations.
Some n

In [17]:
print(generate)

{'Train': {'Benign': 0, 'Malignancy': 0}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}}
